# Engery Charts API
*Insert Lambda Function Name*

In [2]:
import json
import pandas as pd
import datetime as dt
import requests
import psycopg2
import os
import boto3
from psycopg2.extras import execute_values
from dotenv import load_dotenv

In [3]:
load_dotenv('.env', override=True)

# Get Environment Variables
endpoint = os.getenv('ENDPOINT_DL')
db_name = os.getenv('DB_NAME_DL')
username = os.getenv('USERNAME_DL')
password = os.getenv('PASSWORD_DL')
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID_INTERNAL')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY_INTERNAL')
bucket_name = os.getenv('BUCKET_NAME')



In [ ]:

def upload_dataframe_to_db(df, table_name, conn):
    cur = conn.cursor()
    
    # Replace invalid characters in column names and convert index to a column
    df = df.copy()
    df.index.name = 'timestamp'
    df.reset_index(inplace=True)
    df.columns = [col.lower().replace(" ", "_").replace("-", "_") for col in df.columns]
    
    # Generate CREATE TABLE query
    column_defs = ', '.join([
        f"{col} {'timestamp' if col == 'timestamp' else ('text' if col == 'unit' else 'float')}"
        for col in df.columns
    ])
    create_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({column_defs});"
    cur.execute(create_query)

    # INSERT query
    placeholders = ', '.join(['%s'] * len(df.columns))
    insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
    
    # Convert NaN to None for SQL compatibility
    values = [tuple(None if pd.isna(x) else x for x in row) for row in df.values]
    cur.executemany(insert_query, values)
    
    cur.close()

def upload_dataframe_to_bucket(df, foldername,s3 ,bucket_name):
    today = str(dt.datetime.today().date())
    key = foldername + '/' + foldername + '_' + today + '.json'  # Key = path in the bucket

    data = df.reset_index().to_json(orient="records", date_format="iso")

    s3.put_object(
        Bucket=bucket_name,
        Key=key,
        Body=json.dumps(data),
        ContentType='application/json'
    )

def fetch_cbet_data(date: str) -> dict:
    base_url = "https://api.energy-charts.info/cbet"
    params = {
        "country": "ch",
        "start": date
    }
    headers = {
        'accept': 'application/json'
    }

    response = requests.get(base_url, params=params, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error during requesting CBET-Data for {date}: {response.status_code}")
        return {}
    

def extract_cbet_data(json_data: dict) -> pd.DataFrame:
    idx = pd.to_datetime(json_data["unix_seconds"], unit="s", utc=True)
    idx.name = "timestamp"

    frames = []

    for country in json_data.get("countries", []):
        values = country.get("data", [])
        if len(values) != len(idx):
            # length guard – optional but helpful for debugging mismatches
            raise ValueError(
                f"Length mismatch for {country.get('name')}: "
                f"{len(values)} values vs {len(idx)} timestamps"
            )
        df = pd.DataFrame(
            {"value": values, "country": country.get("name")},
            index=idx
        )
        frames.append(df)

    return pd.concat(frames).reset_index()

def insert_cbet_data_to_db(df_cbet, table_name, conn):
    cur = conn.cursor()
    df_cbet = df_cbet.fillna(0)

    column_defs = ', '.join([
        f"{col} {'timestamp' if col == 'timestamp' else ('text' if col == 'country' else 'float')}"
        for col in df_cbet.columns
        ])

    create_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({column_defs});"
    cur.execute(create_query)

    insert_query = f"""
    INSERT INTO {table_name} (timestamp, country, value)
    VALUES %s
    """
    data_tuples = [
        (row['timestamp'], row['country'], row['value'])
        for _, row in df_cbet.iterrows()
    ]

    try:
        execute_values(cur, insert_query, data_tuples)
        conn.commit()
        print("CBET-Data successfully implemented.")
    except Exception as e:
        print(f"Error in implementing CBET-Data: {e}")
        conn.rollback()
    finally:
        cur.close()

def lambda_handler_historic():
    date = dt.datetime.today().date()
    start_date = str(date-dt.timedelta(days=1))

    api_result_power = requests.get(f'https://api.energy-charts.info/public_power?country=ch&start={start_date}')
    api_result_prices = requests.get(f'https://api.energy-charts.info/price?bzn=CH&start={start_date}')

    api_response_power = api_result_power.json()
    api_response_prices = api_result_prices.json()

    cbet_json = fetch_cbet_data(start_date)

    api_df_power = pd.DataFrame(api_response_power["production_types"])
    api_df_power_t = pd.DataFrame(api_df_power['data'].tolist()).T
    api_df_power_t.columns = api_df_power['name']
    api_df_power_t.index = pd.to_datetime(api_response_power["unix_seconds"], unit='s', utc=True)

    api_df_price = pd.DataFrame(api_response_prices["price"])
    api_df_price['Unit'] = api_response_prices["unit"]
    api_df_price.columns = ['Price', 'Unit']
    api_df_price.index = pd.to_datetime(api_response_prices["unix_seconds"], unit='s', utc=True)

    api_df_cbet = extract_cbet_data(cbet_json)

    try:
        print("Connecting to DB & Bucket...")
        conn = psycopg2.connect(
            host=endpoint,
            dbname=db_name,
            user=username,
            password=password
        )
        conn.set_session(autocommit=True)

        print("Connection to DB successful.")

        s3 = boto3.client('s3',
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key
            )

        print("Connection to Bucket successful.")
                
        # Upload DataFrames
        upload_dataframe_to_db(api_df_power_t, "tbl_energy_production_data", conn)
        upload_dataframe_to_db(api_df_price, "tbl_energy_price_data", conn)
        insert_cbet_data_to_db(api_df_cbet, "tbl_energy_cbet_data", conn)
        
        

        upload_dataframe_to_bucket(api_df_power_t, "energy_production",s3,bucket_name)
        upload_dataframe_to_bucket(api_df_price, "energy_price",s3,bucket_name)
        upload_dataframe_to_bucket(api_df_cbet, "energy_cbet",s3,bucket_name)

        conn.close()
        print("Data uploaded and connection closed.")

    except Exception as e:
        print("Error:", e)

    

    


    return {
        'statusCode': 200,
        'body': json.dumps("Inport succesfully")
    }

lambda_handler_historic()

Connecting to DB & Bucket...
Connection to DB successful.
Connection to Bucket successful.
CBET-Data successfully implemented.
Data uploaded and connection closed.


{'statusCode': 200, 'body': '"Inport succesfully"'}